In [1]:
import pathlib
import warnings

import pandas as pd
import statsmodels.formula.api as smf

warnings.filterwarnings("ignore")  # Ignore all warnings
warnings.simplefilter("ignore")  # Additional suppression method

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm
# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

In [2]:
profile_dict = {
    "organoid_fs": {
        "input_profile_path": pathlib.Path(
            root_dir, "5.EDA/results/linear_modeling/organoid_fs.parquet"
        ),
        "metadata_columns": [
            "patient",
            "object_id",
            "unit",
            "dose",
            "treatment",
            "Target",
            "Class",
            "image_set",
            "Well",
            "Therapeutic Categories",
            "single_cell_count",
        ],
    },
    "single_cell_fs": {
        "input_profile_path": pathlib.Path(
            root_dir, "5.EDA/results/linear_modeling/sc_fs.parquet"
        ),
        "metadata_columns": [
            "patient",
            "object_id",
            "unit",
            "dose",
            "treatment",
            "Target",
            "Class",
            "image_set",
            "Well",
            "Therapeutic Categories",
            "parent_organoid",
        ],
    },
}

## Filter significant features
pvalue threshold is set to 0.05 - statistically significant features    
rsquared threshold is set to 0.5 - the explained variance is at least 50% of the total variance    
rsquared adjusted threshold is set to positive values - the model performs better than the mean    


### Single Cell

In [3]:
df = pd.read_parquet(
    profile_dict["single_cell_fs"]["input_profile_path"],
)
print(df.shape)

(45981, 14)


In [4]:
pvalue_threshold_max = 0.05
rsquared_threshold_min = 0.5
rsquared_adj_threshold_min = 0
coefficient_threshold_min = 1

In [5]:
# filter significant features
df_filtered = df[
    (df["pvalue"] < pvalue_threshold_max)
    & (df["rsquared"] > rsquared_threshold_min)
    & (df["rsquared_adj"] > rsquared_adj_threshold_min)
    & (df["coefficient"].abs() > coefficient_threshold_min)
].copy()
print(df_filtered.shape)
df_filtered.head()

(11, 14)


,patient,treatment,feature,rsquared,rsquared_adj,fvalue,pvalue,coefficient,intercept,Feature_type,Compartment,Channel,Measurement,Extra_info
14177,NF0018,Copanlisib,Colocalization_Cell_DNAMito_MINK2,0.504153,0.500710,146.412102,1.078662e-23,59.369253,-1.052110e-15,Colocalization,Cell,DNAMito,MINK2,None
14179,NF0018,Copanlisib,Intensity_Cell_AGP_CMIX,0.569113,0.566121,190.194580,4.133556e-28,80.036130,2.706169e-16,Intensity,Cell,AGP,CMIX,None
14180,NF0018,Copanlisib,Intensity_Cell_AGP_CMIY,0.587496,0.584631,205.087513,1.763058e-29,11.835125,9.194034e-17,Intensity,Cell,AGP,CMIY,None
14181,NF0018,Copanlisib,Intensity_Cell_AGP_CMIZ,0.595828,0.593021,212.283801,4.029314e-30,18.708212,1.031618e-16,Intensity,Cell,AGP,CMIZ,None
14204,NF0018,Copanlisib,Granularity_Cell_AGP_GRANULARITY1,0.887865,0.887086,1140.161933,2.684918e-70,-44.175199,6.980527e-15,Granularity,Cell,AGP,GRANULARITY1,None


In [6]:
df_filtered["treatment"].unique()

array(['Copanlisib', 'Selumetinib'], dtype=object)

In [7]:
df_filtered["patient"].unique()

array(['NF0018', 'NF0030'], dtype=object)

In [8]:
df_filtered["feature"].unique()

array(['Colocalization_Cell_DNAMito_MINK2', 'Intensity_Cell_AGP_CMIX',
       'Intensity_Cell_AGP_CMIY', 'Intensity_Cell_AGP_CMIZ',
       'Granularity_Cell_AGP_GRANULARITY1',
       'Granularity_Cell_AGP_GRANULARITY2',
       'Intensity_Cytoplasm_AGP_CMIX',
       'Granularity_Cytoplasm_AGP_GRANULARITY1',
       'Colocalization_Nuclei_DNAMito_MINK2',
       'Colocalization_Cytoplasm_AGPMito_MINK2'], dtype=object)

### Organoid

In [9]:
df = pd.read_parquet(
    profile_dict["organoid_fs"]["input_profile_path"],
)
print(df.shape)

(18876, 14)


In [10]:
pvalue_threshold_max = 0.05
rsquared_threshold_min = 0.4
rsquared_adj_threshold_min = 0
coefficient_threshold_min = 1

In [11]:
# filter significant features
df_filtered = df[
    (df["pvalue"] < pvalue_threshold_max)
    & (df["rsquared"] > rsquared_threshold_min)
    & (df["rsquared_adj"] > rsquared_adj_threshold_min)
    & (df["coefficient"].abs() > coefficient_threshold_min)
].copy()
print(df_filtered.shape)
df_filtered.head()

(133, 14)


,patient,treatment,feature,rsquared,rsquared_adj,fvalue,pvalue,coefficient,intercept,Feature_type,Compartment,Channel,Measurement,Extra_info
89,NF0014,Mirdametinib,Intensity_Organoid_ER_STDINTENSITY,0.447425,0.423400,18.623301,0.000256,-1.745595,1.536424e-16,Intensity,Organoid,ER,STDINTENSITY,None
90,NF0014,Mirdametinib,Intensity_Organoid_ER_STDINTENSITYEDGE,0.402125,0.376130,15.469554,0.000664,-1.665961,2.317867e-17,Intensity,Organoid,ER,STDINTENSITYEDGE,None
143,NF0014,Fimepinostat,AreaSizeShape_Organoid_VOLUME,0.456506,0.427901,15.958957,0.000775,-2.108811,5.228555e-16,AreaSizeShape,Organoid,None,VOLUME,None
146,NF0014,Fimepinostat,AreaSizeShape_Organoid_EQUIVALENTDIAMETER,0.628190,0.608621,32.101402,0.000018,-3.092539,6.538228e-17,AreaSizeShape,Organoid,None,EQUIVALENTDIAMETER,None
154,NF0014,Fimepinostat,Colocalization_Organoid_AGPER_MAXMANDERSCOEFFC...,0.403456,0.372059,12.850147,0.001976,-2.665425,4.232982e-14,Colocalization,Organoid,AGPER,MAXMANDERSCOEFFCOSTESM1,None


In [12]:
df_filtered["treatment"].unique()

array(['Mirdametinib', 'Fimepinostat', 'Staurosporine', 'Binimetinib',
       'Copanlisib', 'Cabozantinib', 'Onalespib', 'Rapamycin', 'Digoxin',
       'Ketotifen', 'Trametinib', 'Linsitinib', 'Imatinib', 'Everolimus',
       'Selumetinib'], dtype=object)

In [13]:
df_filtered["patient"].unique()

array(['NF0014', 'NF0016', 'NF0018', 'NF0021', 'NF0030', 'SARCO219'],
      dtype=object)

In [14]:
df_filtered["feature"].unique()

array(['Intensity_Organoid_ER_STDINTENSITY',
       'Intensity_Organoid_ER_STDINTENSITYEDGE',
       'AreaSizeShape_Organoid_VOLUME',
       'AreaSizeShape_Organoid_EQUIVALENTDIAMETER',
       'Colocalization_Organoid_AGPER_MAXMANDERSCOEFFCOSTESM1',
       'Colocalization_Organoid_DNAAGP_MEANCORRELATIONCOEFF',
       'Colocalization_Organoid_DNAAGP_MINMANDERSCOEFFCOSTESM1',
       'Colocalization_Organoid_DNAER_MEDIANCORRELATIONCOEFF',
       'Colocalization_Organoid_DNAER_MEDIANMANDERSCOEFFCOSTESM1',
       'Intensity_Organoid_DNA_CMIX',
       'Intensity_Organoid_DNA_INTEGRATEDINTENSITYEDGE',
       'Intensity_Organoid_DNA_LOWERQUARTILEINTENSITY',
       'Intensity_Organoid_DNA_MEANINTENSITYEDGE',
       'Intensity_Organoid_ER_DIFFZ',
       'Intensity_Organoid_ER_INTEGRATEDINTENSITY',
       'Intensity_Organoid_ER_INTEGRATEDINTENSITYEDGE',
       'Intensity_Organoid_ER_LOWERQUARTILEINTENSITY',
       'Intensity_Organoid_ER_MEANINTENSITYEDGE',
       'Intensity_Organoid_Mito_CMZ',
  